# W2 Form Extraction - Data Preparation for Fine-tuning

Prepare the training, validation, and test datasets in Bedrock conversation schema format
and set up IAM resources for the fine-tuning job.

In this notebook we will:

- Format the W2 ground truth data into JSONL for Bedrock fine-tuning
- Upload the prepared datasets to S3
- Create the IAM role and policy required by the fine-tuning job

**Prerequisite:** Run `01_base_model_eval.ipynb` first to download the dataset and upload images to S3.

## Environment Setup

In [ ]:
import warnings
warnings.filterwarnings("ignore")

from util import *

clients = get_aws_clients()
session    = clients["session"]
s3_client  = clients["s3"]
account_id = clients["account_id"]

In [ ]:
%store -r bucket_name
%store -r train_s3_paths
%store -r val_s3_paths
%store -r test_s3_paths

print(f"Bucket name: {bucket_name}")
print(f"Train samples: {len(train_s3_paths)}")
print(f"Test samples:  {len(test_s3_paths)}")

## Prepare JSONL Datasets

Convert the raw ground truth data into Bedrock conversation schema JSONL files.

In [ ]:
prepare_dataset_jsonl(train_s3_paths, "train.jsonl", account_id)
prepare_dataset_jsonl(test_s3_paths,  "test.jsonl", account_id)

## Upload JSONL Files to S3

In [ ]:
s3_client.upload_file("train.jsonl",      bucket_name, "data/train.jsonl")
s3_client.upload_file("test.jsonl",       bucket_name, "data/test.jsonl")

train_data_uri      = f"s3://{bucket_name}/data/train.jsonl"
test_data_uri       = f"s3://{bucket_name}/data/test.jsonl"

print(f"Training data URI:   {train_data_uri}")
print(f"Test data URI:       {test_data_uri}")

## Create IAM Role for Fine-tuning

In [ ]:
role_arn, role_name, policy_arn = create_iam_resources(
    session, account_id, bucket_name
)

print(f"\nRole ARN:   {role_arn}")
print(f"Role name:  {role_name}")
print(f"Policy ARN: {policy_arn}")

## Save Variables

In [ ]:
%store train_data_uri
%store test_data_uri
%store role_arn
%store role_name
%store policy_arn

print("Variables saved for subsequent notebooks")